In [29]:
# 导入模块
import cv2
import numpy as np

In [30]:
#读取输入图片
imageA = cv2.imread("a.jpg")
imageB = cv2.imread("b.jpg")

In [31]:
# 创建SIFT对象
sift = cv2.SIFT_create()  # 可以改为SIFT
# 提取特征点和描述子
kp1, descrip1 = sift.detectAndCompute(imageA, None)
kp2, descrip2 = sift.detectAndCompute(imageB, None)

In [32]:
# 创建FLANN匹配器
# 指定使用KD树
FLANN_INDEX_KDTREE = 0
indexParams = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
searchParams = dict(checks=50)
# 创建FLANN匹配器实例
flann = cv2.FlannBasedMatcher(indexParams, searchParams)
# 执行knn匹配
match = flann.knnMatch(descrip1, descrip2, k=2)
# 存储匹配点对
good = []
# 过滤特征点
for i, (m, n) in enumerate(match):
    if m.distance < 0.75 * n.distance:
        good.append(m)


In [33]:
# 当筛选后的匹配对大于10时，计算视角变换矩阵
if len(good) > 10:
    src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
    ano_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
    M, mask = cv2.findHomography(src_pts, ano_pts, cv2.RANSAC, 5.0)
    warpImg = cv2.warpPerspective(imageB, np.linalg.inv(M), (imageA.shape[1] + imageB.shape[1], imageB.shape[0]))
    direct = warpImg.copy()
    direct[0:imageA.shape[0], 0:imageB.shape[1]] = imageA


In [34]:
# 计算重叠区域
rows, cols = imageA.shape[:2]
# 从左向右查找第一个非空且与warpImg有重叠的列
for col in range(0, cols):
    if imageA[:, col].any() and warpImg[:, col].any():
        left = col
        print(left)
        break
# 从右向左查找最后一个非空且与warpImg有重叠的列
for col in range(cols - 1, 0, -1):
    if imageA[:, col].any() and warpImg[:, col].any():
        right = col
        print(right)
        break


2017
3999


In [35]:
# 加权处理
res = np.zeros([rows, cols, 3], np.uint8)
for row in range(0, rows):
    for col in range(0, cols):
        if not imageA[row, col].any():  # 如果没有原图，用旋转的填充
            res[row, col] = warpImg[row, col]
        elif not warpImg[row, col].any():
            res[row, col] = imageA[row, col]
        else:
            # 加权混合
            srcImgLen = float(abs(col - left))
            testImgLen = float(abs(col - right))
            alpha = srcImgLen / (srcImgLen + testImgLen)
            res[row, col] = np.clip(imageA[row, col] * (1 - alpha) + warpImg[row, col] * alpha, 0, 255)


In [36]:
# 保存结果
warpImg[0:imageA.shape[0], 0:imageA.shape[1]] = res
cv2.imwrite("a_b.jpg", warpImg)


True